- Ref: https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Qwen3_(4B)-GRPO.ipynb#scrollTo=DkIvEkIIkEyB

Instal unsloth

In [ ]:
import torch
print(torch.__version__)
# uv add "unsloth[cu124-torch251] @ git+https://github.com/unslothai/unsloth.git"

2.5.1+cu124


Pre-fine-tune the model to make GRPO follow the desired output format — this speeds up GRPO.